# BCGNet Demo

The following python toolbox trains a neural network intended for BCG artifact removal in EEG-fMRI datasets. More detail about our method can be found in the paper McIntosh et al. IEEE Trans Biomed Engi at https://ieeexplore.ieee.org/document/9124646

In [1]:
# import commands

import os

from pathlib import Path
from config import get_config
from session import Session

## Path setup

The first step is to set up all the relevant path. Here for the purpose of the demo we will define all path here; however, for custom use it is recommended to set up all path in the yaml file found in the 'config' directory

In [2]:
# get the absolute path to the root directory of the package
d_root = Path(os.getcwd())

# get the absolute path to the directory containing all data
# all dataset should be in EEGLAB format
# here the structure of directory is presumed to be
# d_data / subXX / input_file_naming_format
# where input_file_naming_format is defined in the yaml file
d_data = d_root / 'example_data' / 'raw_data'

# get the absolute path to the directory to save all trained models
# structure of the directory will be
# d_model / model_type / subXX / {model_type}_{time_stamp} / {model_type}_{time_stamp}.index

# (note: depending on TF version, either save in the new TF checkpoint format or old h5 format)
d_model = d_root / 'trained_model' / 'non_cv_model'

# get the absolute path to the directory to save all cleaned dataset
# structure of the directory will be
# d_output / subXX / output_file_naming_format
d_output = d_root / 'cleaned_data' / 'non_cv_data'

# (Optional)
# if the users wish, a dataset used to compare the performance of
# BCGNet can be provided, here a OBS-cleaned dataset is used
# convention is same as the d_data and all dataset
# should be in EEGLAB format

# get the absolute path to the directory containing all data
# cleaned by the alternative method
# here the structure of the directory is also presumed to be
# d_eval / subXX / eval_file_naming_format
d_eval = d_root / 'example_data' / 'obs_cleaned_data'

# (Optional - relevant only if  d_eval is provided)
# define the name of the alternative method
str_eval = 'OBS'

# generate a config (cfg) object from the yaml file
# all hyperparameters are from the paper
cfg = get_config(filename=d_root / 'config' / 'default_config.yaml')

# change all the path (recommended to set these in the yaml file directory)
cfg.d_root = d_root
cfg.d_data = d_data
cfg.d_model = d_model
cfg.d_output = d_output
cfg.d_eval = d_eval
cfg.str_eval = str_eval

Once the user has successfully set up all these variable in the yaml file, it's only needed to execute the following command

In [3]:
# cfg = get_config(d_root / 'config' / 'default_config.yaml')

In [4]:
# get rid of this later
cfg.num_epochs = 3

## Initialize training session

All key hyperparamters relevant to preprocessing and training are set in the yaml file

In [5]:
# provide the name of the subject
str_sub = 'sub34'

# provide the index of the runs to be used for training
# if just a single run, then [1] or [2]
# if multiple runs then [1, 2]

# for a run from sub11 and run index 1
# filename is presumed to be
# subXX_r0X_
vec_idx_run = [1, 2]


# str_arch specifies the type of the model to be used
# if str_arch is not provided then the default model (same as paper)
# is used. If user wants to define their own model, example on how to do it
# can be found in models/gru_arch_000.py, the only caveat is that 
# the name of the file and class name has to be same as the type of the model
# e.g. gru_arch_000

# random_seed is set to ensure that the splitting of entire dataset into
# training, validation and test sets is always the same, useful for model
# selection

# verbose sets the verbosity of Keras during model training
# 0=silent, 1=progress bar, 2=one line per epoch

# overwrite specifies whether or not to overwrite existing cleaned data

# cv_mode specifies whether or not to use cross validation mode
# more on this later
s1 = Session(str_sub=str_sub, vec_idx_run=vec_idx_run, str_arch='default_rnn_model',
             random_seed=1997, verbose=2, overwrite=False, cv_mode=cv_mode, num_fold=5, cfg=cfg)

## Prepare for training

In [6]:
# loads all dataset
s1.load_all_dataset()

# preform preprocessing of all dataset and initialize model
s1.prepare_training()

Reading /home/jyao/Local/gitprojects/bcgremover/example_data/eeg_fmri_with_aux_channel/sub34/sub34_r01_rs.fdt
Reading 0 ... 156651  =      0.000 ...   313.302 secs...
Reading /home/jyao/Local/gitprojects/bcgremover/example_data/obs_cleaned_data/sub34/sub34_r01_rmbcg.fdt
Reading 0 ... 156651  =      0.000 ...   313.302 secs...

Resample from 500.0 Hz to 100 Hz

Rejecting 0 epochs out of a total of 104
104 epochs remaining...

Reading /home/jyao/Local/gitprojects/bcgremover/example_data/eeg_fmri_with_aux_channel/sub34/sub34_r02_rs.fdt
Reading 0 ... 156651  =      0.000 ...   313.302 secs...
Reading /home/jyao/Local/gitprojects/bcgremover/example_data/obs_cleaned_data/sub34/sub34_r02_rmbcg.fdt
Reading 0 ... 156651  =      0.000 ...   313.302 secs...

Resample from 500.0 Hz to 100 Hz

Rejecting 0 epochs out of a total of 104
104 epochs remaining...

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param 

## Model training and generating cleaned dataset

In [7]:
# train the model
s1.train()

# generate cleaned dataset
s1.clean()

Epoch 1/3
146/146 - 9s - loss: 0.6776 - val_loss: 0.5228
Epoch 2/3
146/146 - 8s - loss: 0.4925 - val_loss: 0.4458
Epoch 3/3
146/146 - 8s - loss: 0.4332 - val_loss: 0.3940


## Evaluating the performance

In [8]:
# Evaluate the performance of the model in terms of RMS and
# ratio of band power of cleaned dataset in delta, theta 
# and alpha bands compared to the raw data

# mode specifies which set to evaluate the performance on
# mode='train' evaluates on training set
# mode='valid' evaluates on validation set
# mode='test' evaluates on test set
s1.evaluate(mode='test')


#############################################
#                  Results                  #
#############################################

RMS VALUES: RUN 1, TEST SET
RMS Raw: 25.165010030456482
RMS OBS: 16.192650634658857
RMS BCGNet: 14.457144384494656

FREQUENCY BAND POWER REDUCTION: RUN 1
Results for Delta band
Type      Total Power    Ratio to Raw
------  -------------  --------------
OBS           238.417        0.207802
BCGNet        218.301        0.190269

Results for Theta band
Type      Total Power    Ratio to Raw
------  -------------  --------------
OBS           175.827        0.488086
BCGNet        144.432        0.400934

Results for Alpha band
Type      Total Power    Ratio to Raw
------  -------------  --------------
OBS           177.248        0.731969
BCGNet        139.724        0.577009

________________________________________

RMS VALUES: RUN 2, TEST SET
RMS Raw: 23.75340627138015
RMS OBS: 15.47257687042304
RMS BCGNet: 14.523504431809883

FREQUENCY BAND POWER R

## Saving trained model and cleaned dataset

In [9]:
# save trained model
s1.save_model()

# save cleaned data in .mat files
# the saved .mat file has one field 'data' which contains the 
# n_channel by n_time_stamp matrix holding all cleaned data
s1.save_data()

# alternatively, save cleaned data in Neuromag .fif format 
# (note that EEEGLAB support for .fif format is limited)
# s1.save_dataset()

## Cross validation mode

Alternatively, if cross validation is deemed necessary, the users can set up a cross validation style session via the following command

In [10]:
# first change the output and model directory
d_model = d_root / 'trained_model' / 'cv_model'
d_output = d_root / 'cleaned_data' / 'cv_data'
cfg.d_model = d_model
cfg.d_output = d_output

# it is recommended for user to set the num_fold argument,
# which specifies the number of cross validation folds
# in which case, percentage of test set and validation set data
# will be set to 1/num_fold and remaining data will be the training set
# e.g.
s2 = Session(str_sub=str_sub, vec_idx_run=vec_idx_run, str_arch='default_rnn_model',
             random_seed=1997, verbose=2, overwrite=True, cv_mode=True, num_fold=5, cfg=cfg)

# otherwise the number of cross validation folds will be inferred from
# percentage of test set data set in the config yaml file via 1/per_test
# s2 = Session(str_sub=str_sub, vec_idx_run=vec_idx_run, str_arch='default_rnn_model',
#                     random_seed=1997, verbose=2, overwrite=True,
#                     cv_mode=True, cfg=cfg)

Remaining commands are the same

In [13]:
s2.load_all_dataset()
s2.prepare_training()

Reading /home/jyao/Local/gitprojects/bcgremover/example_data/eeg_fmri_with_aux_channel/sub34/sub34_r01_rs.fdt
Reading 0 ... 156651  =      0.000 ...   313.302 secs...
Reading /home/jyao/Local/gitprojects/bcgremover/example_data/obs_cleaned_data/sub34/sub34_r01_rmbcg.fdt
Reading 0 ... 156651  =      0.000 ...   313.302 secs...

Resample from 500.0 Hz to 100 Hz

Rejecting 0 epochs out of a total of 104
104 epochs remaining...

Reading /home/jyao/Local/gitprojects/bcgremover/example_data/eeg_fmri_with_aux_channel/sub34/sub34_r02_rs.fdt
Reading 0 ... 156651  =      0.000 ...   313.302 secs...
Reading /home/jyao/Local/gitprojects/bcgremover/example_data/obs_cleaned_data/sub34/sub34_r02_rmbcg.fdt
Reading 0 ... 156651  =      0.000 ...   313.302 secs...

Resample from 500.0 Hz to 100 Hz

Rejecting 0 epochs out of a total of 104
104 epochs remaining...

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Para

In [14]:
s2.train()
s2.clean()



Training the 1-th fold
Epoch 1/3
138/138 - 10s - loss: 0.6755 - val_loss: 0.5665
Epoch 2/3
138/138 - 9s - loss: 0.4865 - val_loss: 0.4669
Epoch 3/3
138/138 - 8s - loss: 0.4262 - val_loss: 0.4345


Training the 2-th fold
Epoch 1/3
138/138 - 9s - loss: 0.7008 - val_loss: 0.5370
Epoch 2/3
138/138 - 8s - loss: 0.5272 - val_loss: 0.4558
Epoch 3/3
138/138 - 8s - loss: 0.4735 - val_loss: 0.4130


Training the 3-th fold
Epoch 1/3
140/140 - 10s - loss: 0.6974 - val_loss: 0.5673
Epoch 2/3
140/140 - 8s - loss: 0.5338 - val_loss: 0.4850
Epoch 3/3
140/140 - 8s - loss: 0.4687 - val_loss: 0.4086


Training the 4-th fold
Epoch 1/3
140/140 - 9s - loss: 0.6823 - val_loss: 0.5953
Epoch 2/3
140/140 - 8s - loss: 0.5101 - val_loss: 0.4867
Epoch 3/3
140/140 - 9s - loss: 0.4508 - val_loss: 0.4356


Training the 5-th fold
Epoch 1/3
140/140 - 10s - loss: 0.6727 - val_loss: 0.5755
Epoch 2/3
140/140 - 8s - loss: 0.5171 - val_loss: 0.5171
Epoch 3/3
140/140 - 8s - loss: 0.4661 - val_loss: 0.4469


Training the 6-

In [15]:
s2.evaluate(mode='test')


#############################################
#                  Results                  #
#############################################

Cross Validation Fold 1/6


RMS VALUES: RUN 1, TEST SET
RMS Raw: 25.281115434505427
RMS OBS: 15.050788887822236
RMS BCGNet: 14.287429670431358

FREQUENCY BAND POWER REDUCTION: RUN 1
Results for Delta band
Type      Total Power    Ratio to Raw
------  -------------  --------------
OBS           207.441        0.172372
BCGNet        229.941        0.191068

Results for Theta band
Type      Total Power    Ratio to Raw
------  -------------  --------------
OBS           157.355        0.414781
BCGNet        140.294        0.36981

Results for Alpha band
Type      Total Power    Ratio to Raw
------  -------------  --------------
OBS           141.275        0.589403
BCGNet        133.672        0.557687

________________________________________

RMS VALUES: RUN 2, TEST SET
RMS Raw: 24.65589743112266
RMS OBS: 14.454462250628357
RMS BCGNet: 14.44457324691

In [16]:
s2.save_model()
s2.save_data()